`requirements/gpu.txt`

In [ ]:
import pandas as pd, gc, itertools
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit_experiments.library import QuantumVolume
from collections import defaultdict, deque
from calibration_helpers import load_payload

In [ ]:
def noisy_simulator_from_payload(payload):
    '''Creates a noisy AerSimulator from the noise model and coupling map in the payload.'''
    
    noise_model = NoiseModel.from_dict(payload["noise_model"])
    coupling_map = payload.get("coupling_map")
    return AerSimulator(
        noise_model=noise_model,
        coupling_map=coupling_map,
        basis_gates=noise_model.basis_gates,
        method="automatic",
        device="GPU",
    )


def run_qv_for_subset(backend, ideal_backend, subset, shots=2000, trials=100, seed=42):
    '''Runs the Quantum Volume experiment for a given subset of qubits and returns the results.'''

    exp = QuantumVolume(
        physical_qubits=list(subset),
        trials=trials,
        seed=seed,
        simulation_backend=ideal_backend,
    )

    exp.set_run_options(shots=shots)

    exp.set_transpile_options(
        coupling_map=backend.coupling_map, # THIS FIXED THE ISSUE I WAS TRYING TO FIX FOR DAYS
        layout_method="trivial",
        routing_method="basic",
        optimization_level=0,
        seed_transpiler=seed,
        num_processes=0,
    )

    return exp.run(backend).block_for_results()

In [3]:
def build_undirected_adj(coupling_map) -> dict[int, set[int]]:
    """
    Returns adjacency dict: {q: set(neighbors)}
    """
    adj = defaultdict(set)
    for a, b in coupling_map:
        a = int(a); b = int(b)
        adj[a].add(b)
        adj[b].add(a)
    return dict(adj)


def is_connected_subset(subset, adj):
    """Check connectivity of induced subgraph on subset (undirected)."""
    subset = list(subset)
    sset = set(subset)
    start = subset[0]
    seen = {start}
    q = deque([start])
    while q:
        u = q.popleft()
        for v in adj.get(u, ()):
            if v in sset and v not in seen:
                seen.add(v)
                q.append(v)
    return len(seen) == len(subset)


def connected_triples_from_coupling(coupling_map, qubits=None):
    """
    Returns list of triples (sorted tuples) that are connected.
    If qubits is provided, restrict enumeration to those qubits.
    """
    adj = build_undirected_adj(coupling_map)

    if qubits is None:
        nodes = sorted(adj.keys())
    else:
        nodes = sorted(set(qubits))

    triples = []
    for triple in itertools.combinations(nodes, 3):
        if is_connected_subset(triple, adj):
            triples.append(triple)
    return triples

In [4]:
def run_qv_over_connected_triples(calibration_path, shots=2000, trials=100, seed=42, restrict_qubits=None, limit=None):
    payload = load_payload(calibration_path)
    coupling_map = payload.get("coupling_map")

    triples = connected_triples_from_coupling(coupling_map, qubits=restrict_qubits)

    if limit is not None:
        triples = triples[:limit]

    backend = noisy_simulator_from_payload(payload)
    ideal_backend = AerSimulator(method="statevector", device="GPU")

    summaries = []
    for i, triple in enumerate(triples, 1):
        print(f"[{i}/{len(triples)}] running QV on triple {triple}")

        exp_data = run_qv_for_subset(
            backend=backend,
            ideal_backend=ideal_backend,
            subset=triple,
            shots=shots,
            trials=trials,
            seed=seed,
        )

        df = exp_data.analysis_results(dataframe=True)

        hop_row = df[df["name"] == "mean_HOP"].iloc[0]

        mean_hop = hop_row["value"].nominal_value
        hop_err  = hop_row["value"].std_dev

        summaries.append((triple, mean_hop, hop_err))

        del exp_data, df
        gc.collect()

    return summaries


In [5]:
cal_path = "calibrations/ibm_marrakesh/20260129_101824.json"

# run on all connected triples
all_triple_expdata = run_qv_over_connected_triples(
    cal_path,
    shots=100,
    trials=100,
    seed=42,
    # limit=20,
)

/tmp/ipykernel_1663/418988160.py:11: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


[1/244] running QV on triple (0, 1, 2)
[2/244] running QV on triple (1, 2, 3)
[3/244] running QV on triple (2, 3, 4)
[4/244] running QV on triple (2, 3, 16)
[5/244] running QV on triple (3, 4, 5)
[6/244] running QV on triple (3, 4, 16)
[7/244] running QV on triple (3, 16, 23)
[8/244] running QV on triple (4, 5, 6)
[9/244] running QV on triple (5, 6, 7)
[10/244] running QV on triple (6, 7, 8)
[11/244] running QV on triple (6, 7, 17)
[12/244] running QV on triple (7, 8, 9)
[13/244] running QV on triple (7, 8, 17)
[14/244] running QV on triple (7, 17, 27)
[15/244] running QV on triple (8, 9, 10)
[16/244] running QV on triple (9, 10, 11)
[17/244] running QV on triple (10, 11, 12)
[18/244] running QV on triple (10, 11, 18)
[19/244] running QV on triple (11, 12, 13)
[20/244] running QV on triple (11, 12, 18)
[21/244] running QV on triple (11, 18, 31)
[22/244] running QV on triple (12, 13, 14)
[23/244] running QV on triple (13, 14, 15)
[24/244] running QV on triple (14, 15, 19)
[25/244] runni

In [5]:
import qiskit_aer
from qiskit_aer.noise import NoiseModel
print("Aer:", qiskit_aer.__version__)
print([m for m in dir(NoiseModel) if "map" in m.lower() or "label" in m.lower() or "remap" in m.lower()])


Aer: 0.15.1
['_instruction_names_labels']


In [6]:
df = pd.DataFrame(all_triple_expdata, columns=["triple", "mean_HOP", "hop_error"])
df = df.sort_values("mean_HOP", ascending=False).reset_index(drop=True)

out_csv = "results/ibm_marrakesh_qv_all_triples.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)

Saved: results/ibm_marrakesh_qv_all_triples.csv


In [ ]:
payload = load_payload("calibrations/ibm_marrakesh/20260129_101824.json")
coupling_map = payload.get("coupling_map")
triples = connected_triples_from_coupling(coupling_map, qubits=None)
print(f"Found {len(triples)} connected triples:")